In [1]:
# !wget -O faces_dataset.tar.gz "https://www.dropbox.com/scl/fi/7dv71y3nxrcdrpmwntr8e/faces_aligned_small_mirrored_co_aligned_cropped_cleaned.tar.gz?rlkey=h03r92h1mdr9yet2tkqosqq1k&dl=1"

In [ ]:
# import tarfile

# with tarfile.open("faces_dataset.tar.gz", "r:gz") as tar:
#     tar.extractall("faces_dataset")

In [ ]:
# import os
# file_names = os.listdir("/kaggle/working/faces_dataset/faces_aligned_small_mirrored_co_aligned_cropped_cleaned/M")
# len(file_names)

17673

In [ ]:
import sys
!git clone https://github.com/USERNAME/REPO.git
sys.path.append('/kaggle/working/REPO')

In [ ]:
import torch
import pytorch_lightning as pl

from gda_functions import Config, set_seed_all, DiffusionTrainer, StreamDataModule

# Configure training for Kaggle 2*T4 GPUs
cfg = Config()

use_gpu = torch.cuda.is_available()
n_gpus = torch.cuda.device_count() if use_gpu else 0
strategy = "ddp_notebook" if (use_gpu and n_gpus >= 2) else "auto"


# If no GPU, downgrade batch_size/steps/precision for faster CPU test runs
if not use_gpu:
    cfg.precision = "32-true"
    cfg.steps = min(cfg.steps, 500)
    cfg.batch_size = min(cfg.batch_size, 2048)

set_seed_all(cfg.seed)

# PyTorch Lightning DataModule automatically manages correct device and sharing stream params among workers
datamodule = StreamDataModule(cfg)
model = DiffusionTrainer(cfg)

# Lightning config for multi-GPU (Kaggle 2xT4) or fallback
trainer = pl.Trainer(
    max_steps=cfg.steps,
    accelerator="gpu" if use_gpu else "cpu",
    devices=n_gpus if use_gpu else None,
    strategy=strategy,
    precision=cfg.precision,
    log_every_n_steps=cfg.log_every_n_steps,
    enable_progress_bar=True,
    enable_checkpointing=False,
    gradient_clip_val=cfg.grad_clip_norm,
)

trainer.fit(model, datamodule=datamodule)

# Extract and show learned parameters and data family params
denoiser = model.model  # HalfSpaceDenoiser
state = {k: v.detach().cpu() for k, v in denoiser.state_dict().items()}

print("Learned parameters:")
print("w0:", state.get("w0"))
print("w1:", state.get("w1"))
if getattr(denoiser, "use_extended", False):
    print("w2:", state.get("w2"))
    print("w3:", state.get("w3"))
print("b:", state.get("b"))

# Read the actual training stream parameters if available
if getattr(datamodule, "stream", None) is not None:
    print("\nData family parameters (training stream):")
    print("mu0:", datamodule.stream.mu0.detach().cpu())
    print("u:", datamodule.stream.u.detach().cpu())
else:
    print("\nData family parameters not available from datamodule.stream")


ModuleNotFoundError: No module named 'torch'